In [ ]:
!pip install requests
!pip install bs4

In [4]:
#### 利用愛食記,查詢三大餐飲集團旗下各品牌餐廳的相關資訊 ####
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os

### 編寫爬蟲架構--根據搜尋的關鍵字,每次爬取3頁(爬太多頁就會搜到很多錯的)的分店大標題和其內容(一頁最多顯示15個大標題)
def crawler(company,restaurant):
  headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}
  final_list = [] 

  for i in range(3):
    try:    # 有的餐廳比較冷門會查不到半篇
      url = f'https://ifoodie.tw/explore/list/{restaurant}?page={i+1}'
      res = requests.get(url,headers=headers)
      soup = BeautifulSoup(res.text,'html.parser')
      main_tag = soup.select('div[class="jsx-558691085 info-rows"]')  
      
      for tag in main_tag:
        main_article_title = tag.find('div',{'class':"jsx-558691085 title"}).text.split(".")[1].strip()   # 分店大標題
        main_article_url = "https://ifoodie.tw"+tag.find('div',{'class':"jsx-558691085 title"}).find('a').get('href')  # 大標題連結(內有評論和相關食記)
        main_article_score = tag.find('div',{'class':"jsx-1207467136 text"}).text.strip()   # 分店評分
        main_article_reviewcount = tag.find('a',{'class':"jsx-558691085 review-count"}).text.split("則")[0].split("(")[1].strip()  # 分店評論數
        main_article_category = tag.findAll('a',{'class':"jsx-558691085 category"})[1].text.strip()  # 分店菜系

        ## 針對每個大標題,進入連結取得所有評論及相關食記的內容
        # 發現評論和內容是以動態網頁呈現,所以直接從network找尋API,並找到URL的規律
        if restaurant in main_article_title:   # 有些查詢到的分店跟關鍵字根本無關,先過濾掉
          # 從大標題的URL萃取出API有用到的article_id
          article_id = main_article_url.split("restaurant/")[1].split("-")[0]  

          ## 萃取評論文字
          review_api = f'https://ifoodie.tw/api/checkin/?restaurant_id={article_id}&offset=0&limit=100'
          review_dict = json.loads(requests.get(review_api).text)
          review_content = ""

          for row in range(len(review_dict["response"])):
            review = review_dict["response"][row]["message"] 
            # 有些評論是食記,先跳過
            if review_dict["response"][row]["user"]["certified"] == False:   
                review_content = review_content + review + "\n-------------------------------------------\n"
            else:
              continue

          ## 萃取食記URL及文章內容文字
          article_api = f'https://ifoodie.tw/api/restaurant/{article_id}/blogs/?offset=0&limit=100'          
          article_dict = json.loads(requests.get(article_api).text)
          article_info_list = []

          for row2 in range(len(article_dict["response"])):
            # 找到每篇食記的URL
            blog_id = article_dict["response"][row2]["stat"]["blog_id"]
            article_url = f'https://ifoodie.tw/post/{blog_id}'
            
            try:     # 有的食記是外部網站,不適用這種格式的URL,先略過
              # 進入連結並爬取每個食記的文章內容
              res2 = requests.get(article_url)
              soup2 = BeautifulSoup(res2.text,'html.parser')
              article_time = soup2.find('div',{'class':"jsx-3054895173 publish-date"}).text.split("：")[1].strip()
              content = soup2.findAll('p',{'class':"jsx-472180199"})
              article_title = soup2.find('h1',{'class':"jsx-3054895173 post-title"}).text.strip()
              article_content = article_title + "-----------"
              for texttag in content :
                  article_content = article_content + texttag.text.strip() + ","

              # 把文章資訊統整在一個list內
              article_info_list.append(
                  {'文章連結' : article_url,
                  '發布時間' : article_time,
                  '內容' : article_content}
                  )

            except:
              pass

          ## 將所有資訊整理進一個list裡面,以便之後轉換為df
          final_list.append(
            {'來源網站': "愛食記",
            '集團': company,
            '品牌': restaurant,
            '標題': main_article_title,
            '連結' : main_article_url,
            '菜系': main_article_category,
            '評分': main_article_score,
            '留言數': main_article_reviewcount,
            '文章內容': review_content, 
            '食記資訊': article_info_list}
            )

        else:
          continue

    except:
      pass

  ## 將資料轉換成表格呈現
  df = pd.DataFrame(final_list)
  try:   # 有些冷門餐廳查不到資料,會無法生成df
    df.to_csv(f"C:\\Users\\user\\Desktop\\ifoodie\\{company}\\{restaurant}.csv",index=0,encoding='utf-8-sig',header=['來源網站','集團','品牌','標題','連結','菜系','評分','留言數','文章內容','食記資訊'])    # 路徑請視自身環境修改
  except:
    pass


### 輸入關鍵字開始爬蟲
restaurant_list = [{"company":"王品集團","restaurant": ["王品牛排","西堤","陶板屋","原燒","聚北海道鍋物","藝奇","夏慕尼","品田牧場","石二鍋","hot 7","莆田","青花驕","享鴨","丰禾","樂越","12MINI","THE WANG","和牛涮","尬鍋","肉次方"]},
                   {"company":"瓦城泰統","restaurant": ['瓦城','非常泰','1010湘','大心','時時香','月月THAI','YABI KITCHEN']},
                   {"company":"漢來美食","restaurant": ['漢來海港','漢來名人坊','漢來蔬食','漢來軒','翠園粵菜','五梅先生','安那居','上菜','福園台菜','弁慶','精緻海鮮火鍋','糕餅小舖','大廳酒廊','漢來咖啡廳','Pavo','焰·鐵板燒']}]

for n in range(3):
  # 為各個集團創建資料夾存放CSV檔(路徑請視自身環境修改)
  if not os.path.exists(f"C:\\Users\\user\\Desktop\\ifoodie\\{restaurant_list[n]['company']}"):
    os.mkdir(f"C:\\Users\\user\\Desktop\\ifoodie\\{restaurant_list[n]['company']}")

  for restaurant in range(len(restaurant_list[n]["restaurant"])):
    crawler(restaurant_list[n]["company"],restaurant_list[n]["restaurant"][restaurant])    